# Network Analysis
## Crime Network Analysis
### Professor - Michele Coscia

This notebook contains all of the code developed for the Network Analysis Course at ITU. We will be using...**finish**

**Add Description**

Group 10:<br>
Crisanna Cornish (ccor@itu.dk)<br>
Danielle Dequin (ddeq@itu.dk)<br>
Moneeca Abru Iftikhar Latif (abml@itu.dk)<br>
Carl August Wismer (cwis@itu.dk)<br>
Sabrina Fonseca Pereira (sabf@itu.dk)<br>
Maria Do Carmo Madeira Santos Silva Passos de Sousa (mdom@itu.dk)

Created: 27-09-2021<br>
Last Modified: 27-09-2021